In [1]:
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import split, explode,to_date

spark = SparkSession.builder \
    .appName("refine_gkg_counts") \
    .config("spark.driver.memory", "5g")\
    .config("spark.executor.memory", "16g")\
    .getOrCreate()
    # .config("spark.memory.offHeap.enabled",True)\
    # .config("spark.memory.offHeap.size","8g") \

base_dir = '/home/oscar/budasbi-repos/factored-datathon-2024-voyager/bucket_contents/raw/gkg_counts'

gkg_files = []
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".csv"):
            gkg_files.append(os.path.join(root, file))


24/08/19 12:41:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
spark.sparkContext

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
#spark.driver.memoryOverhead
spark.executor.memoryOverhead

AttributeError: 'SparkSession' object has no attribute 'executor'

In [ ]:
if gkg_files:
    gkg_counts = spark.read.csv(gkg_files, header=True, sep='\t',inferSchema=True)
else:
    print("We didn't find any CSV file")


In [ ]:
gkg_counts = gkg_counts.persist()

In [ ]:
gkg_counts.count()#307576

In [ ]:
gkg_counts.printSchema()

In [ ]:
gkg_colums=['date','numarts', 'count_type', 'number', 'object_type', 'geo_type', 'geo_fullname', 'geo_country_code', 'geo_adm1_code', 'geo_lat', 'geo_long','geo_feature_id', 'cameo_event_ids', 'sources', 'source_urls' ]
gkg_counts = gkg_counts.toDF(*gkg_colums)

#geo_type = {1:'Country', 2:'US State', 3:'US City', 4: 'World City', 5: 'World State', 0: 'Not Assigned'}
#geo_country_code resolve with 
#geo_adm1_code -> https://github.com/mysociety/gaze/blob/master/data/us-states.csv
#Explote cameo_event_ids

In [ ]:
gkg_counts = gkg_counts.withColumn('date', to_date('date', 'yyyyMMdd'))

In [ ]:
#Split Cameo_event_ids
gkg_counts = gkg_counts.withColumn("cameo_event_ids_split", split(gkg_counts["cameo_event_ids"], ","))
gkg_counts = gkg_counts.withColumn("cameo_event_ids", explode(gkg_counts["cameo_event_ids_split"]))
gkg_counts = gkg_counts.drop("cameo_event_ids_split")

In [ ]:
# num_partitions = 20  # Ajusta este valor según el tamaño de tus datos y recursos disponibles
# df = gkg_counts.repartition(num_partitions)

In [ ]:
# gkg_counts.count()

In [ ]:
#gkg_counts.select('cameo_event_ids').distinct().orderBy('cameo_event_ids', ascending = False).show()

In [ ]:
#gkg_counts.select('cameo_event_ids').distinct().orderBy('cameo_event_ids', ascending = True).show()

In [ ]:
# df.explain()


In [ ]:
from pyspark.storagelevel import StorageLevel
# Persistir el DataFrame en memoria
df = gkg_counts.persist()

# Eliminar duplicados
df = df.dropDuplicates()

# Contar los registros
##record_count = df_sin_duplicados.count()


In [ ]:
df.count()

Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/oscar/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/oscar/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/oscar/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/home/oscar/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
    self._run_once()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 1871, in _run_once
    event_list = self._selector.select(timeout)


In [ ]:
df_sin_duplicados.explain()

In [ ]:

print(f"El número total de registros únicos es: {record_count}")

# Liberar el DataFrame de la caché después de que ya no sea necesario
df.unpersist()


In [ ]:
record_count = no_duplicates.count()

In [ ]:
gkg_counts.printSchema()

In [ ]:
gkg_counts.createOrReplaceTempView('gkg_counts')
limit10 = spark.sql("select count(*) from gkg_counts ")
limit10.show(truncate=False)